In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'bs4'

- Input a date range and a polygon for area(s) of interest
- Find folders that are within that date range
- Find images that intersect the polygons
- Clip the images that intersect by the polygons and save geotiff of the interecting area

In [10]:
base_url = "https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2"

In [16]:
start_date = "2023-05-01"
end_date = "2023-05-10"

In [26]:
def create_date_url(base_url, input_date):
    year = input_date.strftime("%Y")
    month = input_date.strftime("%m")
    day = input_date.strftime("%d")
    return f"{base_url}/{year}/{month}/{day}"


def get_existing_folders(base_url, start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start_date
    urls = []
    while current_date <= end_date:
        check_url = create_date_url(base_url, current_date)
        response = requests.get(check_url, timeout=10)
        if response.status_code == 200:
            urls.append(check_url)
        current_date += timedelta(days=1)
    return urls


date_urls = get_existing_folders(base_url, start_date, end_date)
date_urls[1:3]

['https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/05/02',
 'https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/05/03']